# sPyTial Dataclass Builder with CnD-core

This notebook demonstrates the new CnD-core based dataclass builder widget that provides **visual, interactive construction** of dataclass instances.

## Key Features

- **Visual Construction**: Uses CnD-core's `structured-input-graph` for interactive building
- **No File I/O**: Direct JavaScript-Python communication via postMessage
- **Real-time Value**: `widget.value` always contains the current dataclass instance
- **Spatial Annotations**: Automatically includes any spatial constraints from decorators

## Installation

Make sure you have ipywidgets installed:

```bash
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```

## Basic Usage

In [1]:
from dataclasses import dataclass
import spytial

In [2]:
@dataclass
class Person:
    name: str = "placeholder"
    age: int = 0
    email: str = "p2"

### Create the Builder Widget

This creates an interactive CnD-core based visual builder:

In [3]:
person_widget = spytial.dataclass_builder(Person)
person_widget

### Instructions:

1. Use the visual interface above to build your `Person` instance
2. Click the **"Export JSON"** button when done
3. The data is automatically captured in `person_widget.value`

In [5]:
# Access the built instance
person = person_widget.value
print(f"Built person: {person}")
print(f"Name: {person.name if person else 'Not built yet'}")

Built person: None
Name: Not built yet


## Debugging Export

If export isn't working, run this cell to check the console logs:

## With Spatial Annotations

The widget automatically includes spatial constraints from sPyTial decorators:

In [ ]:
@dataclass
@spytial.orientation(selector='self', directions=['horizontal'])
class TreeNode:
    value: int = 0
    left: 'TreeNode' = None
    right: 'TreeNode' = None

In [ ]:
tree_widget = spytial.dataclass_builder(TreeNode)
tree_widget

In [ ]:
# The widget respects spatial annotations from decorators
tree = tree_widget.value
print(f"Built tree: {tree}")

## Workflow: Build → Visualize

A common workflow is to build data with the widget, then visualize it:

In [ ]:
# Step 1: Build your data
widget = spytial.dataclass_builder(Person)
widget

In [ ]:
# Step 2: After building and exporting, visualize it
if widget.value:
    spytial.diagram(widget.value)

## Architecture

### How It Works

1. **CnD Spec Generation**: Extracts spatial annotations from decorators and generates YAML spec
2. **Initial Data**: Creates empty dataclass instance and converts to CnD data format
3. **Visual Interface**: Embeds `structured-input-graph` web component in iframe
4. **Export Handler**: JavaScript postMessage sends data back to Python kernel
5. **Value Update**: Automatically converts JSON to dataclass and updates `widget.value`

### No File I/O, No Timeouts

Unlike the old system:
- ❌ No temporary files or export directories
- ❌ No polling or timeout mechanisms
- ✅ Direct kernel communication via `window.Jupyter.notebook.kernel.execute()`
- ✅ Immediate value availability at `widget.value`

### CnD-core Integration

Uses the following CnD components:
- `structured-input-graph`: Visual construction interface
- `JSONDataInstance`: Data instance format
- CnD spec (YAML): Spatial constraint specification

## Comparison: Old vs New

### Old System (dataclass_widget.py)
```python
# Complex file-based system
widget = spytial.dataclass_widget(Person)
# Wait for file export...
# Poll directory...
# Hope timeout doesn't expire...
person = widget.value  # Might be None
```

### New System (dataclass_widget_cnd.py)
```python
# Simple, direct communication
widget = spytial.dataclass_builder(Person)
# Build visually, click Export
person = widget.value  # Immediately available
```